In [41]:
import sys
from nltk.tokenize import sent_tokenize
import pandas as pd
from sqlalchemy import text
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import gensim
from gensim.models import Word2Vec
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
import ast
from nltk.corpus import wordnet

In [42]:
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/jengyuchou/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jengyuchou/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/jengyuchou/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/jengyuchou/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

# Lemmatizer

In [43]:
# function to convert nltk tag to wordnet tag
def nltk_tag_to_wordnet_tag(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:          
        return None

def lemmatize_sentence(sentence):
    #tokenize the sentence and find the POS tag for each token
    nltk_tagged = nltk.pos_tag(nltk.word_tokenize(sentence))  
    #tuple of (token, wordnet_tag)
    wordnet_tagged = map(lambda x: (x[0], nltk_tag_to_wordnet_tag(x[1])), nltk_tagged)
    lemmatized_sentence = []
    for word, tag in wordnet_tagged:
        if tag is None:
            #if there is no available tag, append the token as is
            lemmatized_sentence.append(word)
        else:        
            #else use the tag to lemmatize the token
            lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))
    return " ".join(lemmatized_sentence)

# Lemmatize Before Word Embeddings

In [47]:
sqlite_path = "../reddit_ed_0.6.0.sqlite3"
engine = create_engine(
    f"sqlite:///{sqlite_path}",
    connect_args={"check_same_thread":False},
)

In [54]:
query = f"select subreddit, count(*) from subreddit_submission_metadata group by subreddit;"
subreddits = pd.read_sql_query(text(query),con=engine.connect())['subreddit'].tolist()

In [63]:
def lemmatized_preprocess(text):
    result=[]
    for token in gensim.utils.simple_preprocess(lemmatize_sentence(text)) :
        
        if token not in stopwords.words('english'):
            
            result.append(token)
    return result

In [64]:
lemmatize_sentences = dict()

In [65]:
subreddits_lemmatized_selftext = dict()
for subreddit in subreddits:
    query = f"select selftext from subreddit_submission_metadata where subreddit='{subreddit}';"
    df = pd.read_sql_query(text(query),con=engine.connect())['selftext'].tolist()
    subreddits_lemmatized_selftext[subreddit] = df
    lemmatize_sentences[subreddit] = [lemmatized_preprocess(doc) for doc in df]

In [69]:
for subreddit in subreddits:
    model = Word2Vec(lemmatize_sentences[subreddit], vector_size=128, window=3, min_count=1, workers=4)
    model.build_vocab(lemmatize_sentences[subreddit])
    print(f"{subreddit} vocab size: {len(model.wv)}")
    W2V_EPOCH = 100
    model.train(lemmatize_sentences[subreddit], total_examples=len(lemmatize_sentences[subreddit]), epochs=W2V_EPOCH)
    model.wv.save_word2vec_format(f"./lemmatized/{subreddit}_word2vec.txt", binary=False)
    

AnorexiaNervosa vocab size: 5037
AnorexiaRecovery vocab size: 7725
BingeEatingDisorder vocab size: 3285
BingeEatingRecovery vocab size: 69
BulimiaAndAnaSupport vocab size: 1538
BulimiaRecovery vocab size: 482
EDAnonymous vocab size: 10471
EDRecovery_public vocab size: 3328
EatingDisorderHope vocab size: 1410
EatingDisorders vocab size: 5927
EdAnonymousAdults vocab size: 11682
NotOtherwiseSpecified vocab size: 212
PurgingDisorder vocab size: 323
anorexiaflareuphelp vocab size: 445
bingeeating vocab size: 3370
bulimia vocab size: 6836
eating_disorders vocab size: 8654
edsupport vocab size: 1729


In [70]:
query = f"select selftext from subreddit_submission_metadata;"
selftext = pd.read_sql_query(text(query),con=engine.connect())['selftext'].tolist()

In [71]:
len(selftext)

21161

In [72]:
preprocessed_lemmatized_selftext = [lemmatized_preprocess(doc) for doc in selftext]

In [73]:
model = Word2Vec(preprocessed_lemmatized_selftext, vector_size=128, window=3, min_count=1, workers=4)
model.build_vocab(preprocessed_lemmatized_selftext)
print(f"selftext vocab size: {len(model.wv)}")
W2V_EPOCH = 100
model.train(preprocessed_lemmatized_selftext, total_examples=len(preprocessed_lemmatized_selftext), epochs=W2V_EPOCH)
model.wv.save_word2vec_format(f"./lemmatized/selftext_word2vec.txt", binary=False)

selftext vocab size: 21537


# Word Embeddings

In [20]:
STOP_WORDS = set(stopwords.words('english'))

In [23]:
sqlite_path = "../reddit_ed_0.6.0.sqlite3"
engine = create_engine(
    f"sqlite:///{sqlite_path}",
    connect_args={"check_same_thread":False},
)

In [24]:
query = f"select subreddit, count(*) from subreddit_submission_metadata group by subreddit;"
subreddits = pd.read_sql_query(text(query),con=engine.connect())['subreddit'].tolist()

In [25]:
def preprocess(text):
    result=[]
    for token in gensim.utils.simple_preprocess(text) :
        
        if token not in stopwords.words('english'):
            
            result.append(token)
    return result

In [26]:
subreddits

['AnorexiaNervosa',
 'AnorexiaRecovery',
 'BingeEatingDisorder',
 'BingeEatingRecovery',
 'BulimiaAndAnaSupport',
 'BulimiaRecovery',
 'EDAnonymous',
 'EDRecovery_public',
 'EatingDisorderHope',
 'EatingDisorders',
 'EdAnonymousAdults',
 'NotOtherwiseSpecified',
 'PurgingDisorder',
 'anorexiaflareuphelp',
 'bingeeating',
 'bulimia',
 'eating_disorders',
 'edsupport']

In [27]:
sentences = dict()

In [84]:
subreddits_selftext = dict()
for subreddit in subreddits:
    query = f"select selftext from subreddit_submission_metadata where subreddit='{subreddit}';"
    df = pd.read_sql_query(text(query),con=engine.connect())['selftext'].tolist()
    subreddits_selftext[subreddit] = df
    sentences[subreddit] = [preprocess(doc) for doc in df]

In [97]:
for subreddit in subreddits:
    model = Word2Vec(sentences[subreddit], vector_size=128, window=3, min_count=1, workers=4)
    model.build_vocab(sentences[subreddit])
    print(f"{subreddit} vocab size: {len(model.wv)}")
    W2V_EPOCH = 100
    model.train(sentences[subreddit], total_examples=len(sentences[subreddit]), epochs=W2V_EPOCH)
    model.wv.save_word2vec_format(f"./Word2Vec/{subreddit}_word2vec.txt", binary=False)

AnorexiaNervosa vocab size: 6121
AnorexiaRecovery vocab size: 9369
BingeEatingDisorder vocab size: 3936
BingeEatingRecovery vocab size: 75
BulimiaAndAnaSupport vocab size: 1782
BulimiaRecovery vocab size: 542
EDAnonymous vocab size: 12651
EDRecovery_public vocab size: 4025
EatingDisorderHope vocab size: 1658
EatingDisorders vocab size: 7284
EdAnonymousAdults vocab size: 14064
NotOtherwiseSpecified vocab size: 223
PurgingDisorder vocab size: 365
anorexiaflareuphelp vocab size: 492
bingeeating vocab size: 4066
bulimia vocab size: 8346
eating_disorders vocab size: 10525
edsupport vocab size: 2024


In [98]:
query = f"select selftext from subreddit_submission_metadata;"
selftext = pd.read_sql_query(text(query),con=engine.connect())['selftext'].tolist()

In [99]:
len(selftext)

21161

In [100]:
preprocessed_selftext = [preprocess(doc) for doc in selftext]

In [105]:
model = Word2Vec(preprocessed_selftext, vector_size=128, window=3, min_count=1, workers=4)
model.build_vocab(preprocessed_selftext)
print(f"selftext vocab size: {len(model.wv)}")
W2V_EPOCH = 100
model.train(preprocessed_selftext, total_examples=len(preprocessed_selftext), epochs=W2V_EPOCH)
model.wv.save_word2vec_format(f"./Word2Vec/selftext_word2vec.txt", binary=False)

selftext vocab size: 25282
